<a href="https://colab.research.google.com/github/salvadorhm/sqlalchemy/blob/main/sqlalchemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLAlchemy Introduccion

## Cargar las librerias de sqlachemy

In [ ]:
from sqlalchemy import create_engine # Conexion con las bases de datos
from sqlalchemy import MetaData # Metadatos de las tablas
from sqlalchemy import Table, Column, Integer, String # Tipos de datos
from sqlalchemy import insert, select # Funciones 

# Conexion con la base de datos

In [ ]:
engine = create_engine('sqlite:///foo.db')

# Objeto para definir las tablas

In [ ]:
metadata_obj = MetaData()

# Definicion de la tabla clientes

In [ ]:
clientes = Table(
    'clientes',
    metadata_obj,
    Column('id_cliente', Integer, primary_key=True),
    Column('nombre', String(16), nullable=False),
    Column('email', String(60), nullable=False),
)

# Creacion de las tablas y de la base de datos si no existiera

In [ ]:
metadata_obj.create_all(engine)

# Statement para insertar un nuevo registro usando SQLAlchemy Core

In [ ]:
stmt = insert(clientes).values(nombre='Cliente 1', email="cliente1@email.com")
print(stmt)

INSERT INTO clientes (nombre, email) VALUES (:nombre, :email)


# Conexion con la base de datos y ejecucion del statement.

**Nota:** with permite abrir y cerrar la sesion con la base de datos de forma automatica.



In [ ]:
with engine.connect() as conn:
    result = conn.execute(stmt)

# Insertar multiple registros

In [ ]:
data = [
    {"nombre":"Cliente 2","email":"cliente2@email.com"},
    {"nombre":"Cliente 3","email":"cliente3@email.com"}
    ]
stmt = insert(clientes).values(data)
print(stmt)

INSERT INTO clientes (nombre, email) VALUES (:nombre_m0, :email_m0), (:nombre_m1, :email_m1)


# Conexion con la base de datos para insertar los valores

In [ ]:
with engine.connect() as conn:
    result = conn.execute(stmt)

# Statement para consultar todos los registros con SQLAlchemy Core

In [ ]:
stmt = select(clientes)
print(stmt)

SELECT clientes.id_cliente, clientes.nombre, clientes.email 
FROM clientes


# Conexion con la base de datos y ejecucion del statement.

In [ ]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result.fetchall():
        print(f"{row['id_cliente']} : {row['nombre']} : {row['email']}")

1 : Cliente 1 : cliente1@email.com
2 : Cliente 2 : cliente2@email.com
3 : Cliente 3 : cliente3@email.com
4 : Cliente 1 : cliente1@email.com
5 : Cliente 2 : cliente2@email.com
6 : Cliente 3 : cliente3@email.com


# Statement para consultar un solo registro

In [ ]:
stmt = select(clientes).where(clientes.c.nombre =='Cliente 1')
print(stmt)

SELECT clientes.id_cliente, clientes.nombre, clientes.email 
FROM clientes 
WHERE clientes.nombre = :nombre_1


# Ejecutando el statement

In [ ]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result.fetchall():
        print(f"{row['id_cliente']} : {row['nombre']} : {row['email']}")

1 : Cliente 1 : cliente1@email.com
4 : Cliente 1 : cliente1@email.com


In [20]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(f"{row.id_cliente}:{row.nombre}:{row.email}")

1:Cliente 1:cliente1@email.com
